In [1]:
#http://www.dgis.salud.gob.mx/contenidos/basesdedatos/da_nacimientos_gobmx.html
# importing libraries
from zipfile import ZipFile
import numpy as np
import openpyxl
import pandas as pd

## Setting read_zip function and colums names

In [2]:
def read_zip(year):
    # specifying the zip file directory
    file_name = f"/mnt/d/Mexico_Births/sinac_{year}.zip"
    
    # opening the zip file in READ mode
    with ZipFile(file_name, 'r') as zip:
        # listing all the contents of the zip file
        file_list = zip.filelist
        # getting a DataFrame from the file in the index 0
        data = pd.read_csv(zip.open(file_list[0].filename) )
    return data

In [3]:
# columns names for different year 
columns_names = {
# columns names for 2020
"columns_20" : ["EDAD","ESTADOCONYUGAL","ESCOLARIDAD","CLAVEOCUPACIONHABITUAL","FECHANACIMIENTO",\
"HORANACIMIENTO", "SEXO", "TALLA", "PESO", "CLUES", "ENTIDADFEDERATIVAPARTO", "MUNICIPIOPARTO",\
"LOCALIDADPARTO"], 
# columns names from 2017 to 2019
"columns_19" : ["EDADM","EDOCIVIL","NIV_ESCOL","CVEOCUPHAB","FECH_NACH",\
"HORA_NACH", "SEXOH", "TALLAH", "PESOH", "CLUES", "ENT_NAC", "MPO_NAC",\
"LOC_NAC"],
# columns names for 2016 & 2015
"columns_16" : ["edad_madre","estado_conyugal","escolaridad_madre","ocupacion_habitual_madre","fecha_nacimiento_nac_vivo",\
"hora_nacimiento_nac_vivo", "sexo_nac_vivo", "talla_nac_vivo", "peso_nac_vivo", "clues", "entidad_nacimiento", "municipio_nacimiento",\
"localidad_nacimiento"],
# new columns names
"new_columns": ["edad_madre", "estado_conyugal_madre","escolaridad_madre","ocupacion_madre", "fecha_nacimiento_nac",\
"hora_nacimiento_nac", "sexo_nac", "talla_nac", "peso_nac", "clues", "entidad_nacimiento", "municipio_nacimiento",\
"localidad_nacimiento"]
}

## Reading auxiliar tables

In [4]:
# reading births_colnames file which contains tables to join on main data
file_name = f"/mnt/d/Mexico_Births/births_colnames.xlsx"
aux_data = pd.read_excel(file_name, sheet_name= None) # using sheet_name=None to get all sheets from excel in a dict
aux_sheets = tuple(aux_data.keys())[4:] # taking sheets name to join with births data 
print(aux_sheets)

('estado_conyugal', 'sexo', 'escolaridad', 'ocupacion2020', 'ocupacion2019')


In [5]:
aux_data[aux_sheets[0]] # estado counyugal table

,Clave_2020,Clave_2019_2016,desc_hom_cony
0,5,11,CASADA
1,1,12,SOLTERA
2,3,13,DIVORCIADA
3,2,14,VIUDA
4,4,15,UNIÓN LIBRE
5,6,16,SEPARADA
6,0,88,NO ESPECIFICADO
7,9,99,SE IGNORA


In [6]:
aux_data[aux_sheets[1]] # genero table

,Clave_2020,Clave_2019_2016,desc_hom_sex
0,0,0,NO ESPECIFICADO
1,1,1,HOMBRE
2,2,2,MUJER
3,9,9,SE IGNORA


In [7]:
aux_data[aux_sheets[2]].head() # escolaridad table

,Clave_2020,Clave_2019_2016,desc_hom_esc
0,0,0,NO ESPECIFICADO
1,1,1,NINGUNA
2,31,3,PRIMARIA COMPLETA
3,32,2,PRIMARIA INCOMPLETA
4,51,5,SECUNDARIA COMPLETA


## Processing data from 2015 to 2020

In [8]:
# function that helps to join our data with auxiliar tables
def joining_tables(data, selected_columns, key_, job_id):
    data = pd.merge(data, aux_data[aux_sheets[0]], how="left", left_on=selected_columns[1], right_on=key_)
    data = pd.merge(data, aux_data[aux_sheets[1]], how="left", left_on=selected_columns[6], right_on=key_)
    data = pd.merge(data, aux_data[aux_sheets[2]], how="left", left_on=selected_columns[2], right_on=key_)
    data = pd.merge(data, aux_data[aux_sheets[job_id]], how="left", left_on=selected_columns[3], right_on=key_)
    return data

# changing values on some  columns for 2015
def changing_vales(data):
    vals = [('S.I.','SE IGNORA'),('N.E.','NO ESPECIFICADO')]
    cl = ["estado_conyugal", "escolaridad_madre", "ocupacion_habitual_madre"]
    for c in cl:
        for v in vals:
            data[data[c] == v[0]] = v[1]
    return data


In [9]:
def get_unified_data(year):
    data = read_zip(year) # reading zip file to get a DF

    # deciding which config take
    if year == 2020:
        key_ = f"Clave_{str(year)}" # key to join with DF
        selected_columns = columns_names[f"columns_{str(year)[-2:]}"] # columns names of the year
        job_id = 3 # ID table of the current mother's job
    elif year in [2019, 2018, 2017]:
        selected_columns = columns_names[f"columns_19"]
        key_ = f"Clave_2019_2016"
        job_id = 4
    else:
        selected_columns = columns_names[f"columns_16"]
        key_ = f"Clave_2019_2016"
        job_id = 4

    # selecting columns 
    data = data[selected_columns] 
    
    if year != 2015:
        # joining with auxiliar tables
        data = joining_tables(data, selected_columns, key_, job_id)
        # sorting
        only_columns = [0, -9, -3, -1, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        data = data.iloc[:,only_columns]
    else:
        # changing values
        data = changing_vales(data)

    # renaming columns
    data.columns = columns_names["new_columns"]

    return data

In [10]:
data = get_unified_data(2019)

/tmp/ipykernel_505/2828715252.py:2: DtypeWarning: Columns (7,51,56,63,65) have mixed types.Specify dtype option on import or set low_memory=False.
  data = read_zip(year) # reading zip file to get a DF
/tmp/ipykernel_505/2397339378.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Clave_2019_2016_x'} in the result is deprecated and will raise a MergeError in a future version.
  data = pd.merge(data, aux_data[aux_sheets[job_id]], how="left", left_on=selected_columns[3], right_on=key_)


In [11]:
data.sample(3)

,edad_madre,estado_conyugal_madre,escolaridad_madre,ocupacion_madre,fecha_nacimiento_nac,hora_nacimiento_nac,sexo_nac,talla_nac,peso_nac,clues,entidad_nacimiento,municipio_nacimiento,localidad_nacimiento
1704723,37,SOLTERA,BACHILLERATO O PREPARATORIA COMPLETA,NO REMUNERADO - AMA DE CASA,09/07/2019,11:12:00,1,50,2900,VZSMP000494,30,118,1
149085,29,UNIÓN LIBRE,PROFESIONAL,NO REMUNERADO - ESTUDIANTE,18/04/2019,16:40:00,1,49,2915,CMIMS000104,6,9,1
94079,28,CASADA,PROFESIONAL INCOMPLETO,NO REMUNERADO - AMA DE CASA,25/11/2019,07:44:00,1,47,3150,CCIMS000045,4,3,1


## Processing data from 2015

In [14]:
data = get_unified_data(2015)

/tmp/ipykernel_505/2828715252.py:2: DtypeWarning: Columns (5,6,7,54,60) have mixed types.Specify dtype option on import or set low_memory=False.
  data = read_zip(year) # reading zip file to get a DF


In [16]:
data.sample(3)

,edad_madre,estado_conyugal_madre,escolaridad_madre,ocupacion_madre,fecha_nacimiento_nac,hora_nacimiento_nac,sexo_nac,talla_nac,peso_nac,clues,entidad_nacimiento,municipio_nacimiento,localidad_nacimiento
1692558,39,CASADA,BACHILLERATO O PREPARATORIA COMPLETA,NO REMUNERADO - AMA DE CASA,05/10/2015,13:08,MUJER,49,3590,DGIMS000042,DURANGO,DURANGO,VICTORIA DE DURANGO
1901904,26,UNIÓN LIBRE,BACHILLERATO O PREPARATORIA COMPLETA,NO REMUNERADO - AMA DE CASA,23/11/2015,08:58,HOMBRE,51,3100,GRSSA004350,GUERRERO,HUITZUCO DE LOS FIGUEROA,CIUDAD DE HUITZUCO
1649510,17,UNIÓN LIBRE,BACHILLERATO O PREPARATORIA INCOMPLETA,NO REMUNERADO - ESTUDIANTE,10/10/2015,08:46,MUJER,50,2850,DFSMP003993,DISTRITO FEDERAL,COYOACAN,COYOACAN
